In [139]:
!pip install folium
import folium
import requests
import pandas
import folium.plugins as plugins
from datetime import datetime as dt

arrest_table = pandas.read_csv("https://cmsc320.github.io/files/BPD_Arrests.csv")

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

<ipython-input-139-bbb69139a04e>:12: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str


In [137]:
#This cell creates a new dataframe which only contains arrests where the crime is classified as an assault

#Dropping all rows that have any NA's, as this will lead to errors in code
charges = arrest_table.dropna()
assaults = []

#Checking if each crime is an assault by looking at the charge description and seeing if assault is in it
for index, row in charges.iterrows():
    if 'Assault' in row['chargeDescription']:
        assaults.append(list(row))

#new dataframe that only has crimes that were classified as an assault
assaults = pandas.DataFrame(data=assaults,columns = arrest_table.columns)

In [142]:
#This cell transforms the arrestDate given in the csv into a datetime object, so the dataframe can be split into 
#bins by month/year.
dates = []
dates_a = []

#adding the date column to the arrest_table dataframe
for x in arrest_table['arrestDate']:
    dates.append(dt.strptime(x, '%m/%d/%Y').strftime('%m/%Y'))

arrest_table['Date'] = dates
    
#adding the date column to the assaults dataframe
for x in assaults['arrestDate']:
    dates_a.append(dt.strptime(x, '%m/%d/%Y').strftime('%m/%Y'))

assaults['Date'] = dates_a

# Heat Map of All Crime In Baltimore By Month (1/11-12/12)

### In this Visualization, all crimes in Baltimore are first separated into bins of month/year. Each bin is then plotted as a Heat Map, where the density of crimes, by location in Baltimore, is able to be seen. In this Map, a gradient from blue to red shows low density to high, respectively. From this visualization, it is easy to spot the locations in Baltimore where most of the crimes occur, and how these differ between months. Specifically, it seems that most of the crime is committed in the inner city. The crime also seems to be highly concentrated in this area around the beginning of summer (frames 6-8/18-20) and becomes more sparsely distributed over the entire city as winter approaches (frames 1 and 12 / 13 and 24)

In [144]:
#Plotting a heatmap of all crimes in Baltimore by coords, separated into bins of month/year from 1/11 - 12/12 
#leading to 24 bins

data = []

#first creating the coordinate pair of latitude and longitude for each pair in each bin, then separating
#each bin into its own list of coordinate pairs
for _, d in arrest_table.groupby('Date'):
   data.append([[row['lat'], row['long']] for _, row in d.iterrows()])

#creating the heatmap with the formatted list
map_heat = folium.Map(location=[39.29, -76.61], zoom_start=11.5)
plugins.HeatMapWithTime(data, auto_play=True,max_opacity=0.8,radius = 10).add_to(map_heat)
map_heat

# Heat Map of Assaults In Baltimore By Month (1/11-12/12)

### The same Heat Map as above is created, but only assault crimes are mapped. Assault crimes seem to not follow the same pattern, of increased density in the inner city during summer months, as strongly as aggregate crimes in the city do. 

In [145]:
#Plotting a heatmap of assault crimes in Baltimore by coords, separated into bins of month/year from 1/11 - 12/12 
#leading to 24 bins
data = []

#first creating the coordinate pair of latitude and longitude for each pair in each bin, then separating
#each bin into its own list of coordinate pairs
for _, d in assaults.groupby('Date'):
   data.append([[row['lat'], row['long']] for _, row in d.iterrows()])

#creating the heatmap with the formatted list
map_heat = folium.Map(location=[39.29, -76.61], zoom_start=11.5)
plugins.HeatMapWithTime(data, auto_play=True,max_opacity=0.8,radius = 10).add_to(map_heat)
map_heat